<a href="https://colab.research.google.com/github/k1151msarandega/1st-order/blob/main/1st_order_rank_order_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Architecture:** *1st-order model*

**Encoding Scheme:** *Rank order encoding*

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from snntorch import snn
import time

1. Define the rank order encoding transform

In [ ]:
def rank_order_encoding(x):
    _, ranks = x.sort(descending=True)
    ranks = ranks.float() / (x.size(1) - 1)
    return ranks


2. Define the SNN model using rank order encoding

In [ ]:
class RankOrderSNN(nn.Module):
    def __init__(self):
        super(RankOrderSNN, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = rank_order_encoding(x)  # Apply rank order encoding
        x = self.fc1(x)
        x = self.fc2(x)
        return x


3. Define the training function

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = snn.LossPoisson(targetRate=35)(output, target)  # Use Poisson loss for SNN
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    end_time = time.time()
    print('Training Time: {:.2f} seconds'.format(end_time - start_time))

4. Define the testing function

In [ ]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    start_time = time.time()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    end_time = time.time()
    print('Test Accuracy: {:.2f}%'.format(accuracy))
    print('Testing Time: {:.2f} seconds'.format(end_time - start_time))

5. Set the device (CPU or GPU)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

6. Define the training and testing datasets

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

7. Define the data loaders

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=1)


8. Create an instance of the SNN model

In [ ]:
model = RankOrderSNN().to(device)

9. Define the optimiser

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


10. Training loop

In [ ]:
for epoch in range(1, 10 + 1):
    print('Epoch:', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)